# OpenAI API Prompt Testing Notebook

This notebook allows you to run structured, multi-round prompt tests with the OpenAI API.

**Round 1:**
- Test `template1.txt`, `template2.txt`, `template3.txt`
- For each template, use `theory_1.txt` as the theory variable
- For each template, iterate over all science texts: `text1.txt`, `text2.txt`, `text3.txt`
- Save results in `results/round1/`

**Round 2:**
- Test only `template3.txt`
- For `template3.txt`, iterate over all science texts and all theory files: `theory_1.txt`, `theory_2.txt`, `theory_3.txt`
- Save results in `results/round2/`

In [14]:
import os
import glob
import datetime
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI client
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found. Please create a .env file and add your API key.")
openai.api_key = api_key

# Directory configuration
PROMPTS_DIR = 'prompts'
SCIENCE_TEXT_DIR = 'variables/science_text'
THEORY_DIR = 'variables/theory'
RESULTS_DIR = 'results'
MODEL_ENGINE = 'gpt-4o'  # You can change this to another model if needed

## Utility Functions
Helper functions for file loading, prompt cleaning, and API calls.

In [6]:
def clean_template_content(template_content):
    """
    Remove comment lines (starting with #) and empty lines from template content.
    """
    lines = template_content.split('\n')
    cleaned_lines = []
    for line in lines:
        stripped_line = line.strip()
        if not stripped_line.startswith('#') and stripped_line:
            cleaned_lines.append(line)
    return '\n'.join(cleaned_lines)

def run_openai_test(prompt_text):
    """
    Send a prompt to the OpenAI API and return the response.
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ENGINE,
            messages=[{"role": "user", "content": prompt_text}]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f'API Error: {e}')
        return None

def get_files(directory, pattern="*.txt"):
    """
    Return a sorted list of files matching the pattern in the directory.
    """
    return sorted(glob.glob(os.path.join(directory, pattern)))

## Round 1: Test template1-3, theory_1, all science texts
- For each template (1, 2, 3), use theory_1.txt
- For each template, iterate over all science texts (text1.txt, text2.txt, text3.txt)
- Save results in results/round1/

In [26]:
round1_results_dir = os.path.join(RESULTS_DIR, 'round1')
os.makedirs(round1_results_dir, exist_ok=True)

# Select templates and theory
template_files = [
    os.path.join(PROMPTS_DIR, f'template1.txt') #for i in [1, 2, 3]
]
theory_file = os.path.join(THEORY_DIR, 'theory_1.txt')

# Get all science texts
science_text_files = get_files(SCIENCE_TEXT_DIR, 'science_text*.txt')

for template_path in template_files:
    template_name = os.path.basename(template_path)
    with open(template_path, 'r') as f:
        template_content = f.read()
    cleaned_template = clean_template_content(template_content)
    with open(theory_file, 'r') as f:
        theory_text = f.read()
    for science_path in science_text_files:
        science_name = os.path.basename(science_path)
        with open(science_path, 'r') as f:
            science_text = f.read()
        # Format the prompt
        prompt = cleaned_template.format(text=science_text, theory=theory_text)
        # Call OpenAI API
        result = run_openai_test(prompt)
        # Save result
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        result_filename = f'{template_name[:-4]}_{science_name[:-4]}_theory1_{timestamp}.txt'
        result_path = os.path.join(round1_results_dir, result_filename)
        with open(result_path, 'w', encoding='utf-8') as f:
            f.write(f'PROMPT:\n{"="*30}\n{prompt}\n\nRESULT:\n{"="*30}\n{result}\n')
        print(f'Saved: {result_path}')

API Error: Connection error.
Saved: results/round1/template1_science_text1_theory1_20250706_054642.txt
API Error: Connection error.
Saved: results/round1/template1_science_text2_theory1_20250706_054718.txt
API Error: Connection error.
Saved: results/round1/template1_science_text3_theory1_20250706_054753.txt


## Round 2: Test template3, all science texts, all theories
- Only use template3.txt
- Iterate over all science texts and all theory files
- Save results in results/round2/

In [23]:
round2_results_dir = os.path.join(RESULTS_DIR, 'round2')
os.makedirs(round2_results_dir, exist_ok=True)

template4_path = os.path.join(PROMPTS_DIR, 'template4.txt')
with open(template4_path, 'r') as f:
    template4_content = f.read()
cleaned_template4 = clean_template_content(template4_content)

science_text_files = get_files(SCIENCE_TEXT_DIR, 'science_text*.txt')
theory_files = get_files(THEORY_DIR, 'theory_*.txt')

for science_path in science_text_files:
    science_name = os.path.basename(science_path)
    with open(science_path, 'r') as f:
        science_text = f.read()
    for theory_path in theory_files:
        theory_name = os.path.basename(theory_path)
        with open(theory_path, 'r') as f:
            theory_text = f.read()
        # Format the prompt
        prompt = cleaned_template4.format(text=science_text, theory=theory_text)
        # Call OpenAI API
        result = run_openai_test(prompt)
        # Save result
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        result_filename = f'template4_{science_name[:-4]}_{theory_name[:-4]}_{timestamp}.txt'
        result_path = os.path.join(round2_results_dir, result_filename)
        with open(result_path, 'w', encoding='utf-8') as f:
            f.write(f'PROMPT:\n{"="*30}\n{prompt}\n\nRESULT:\n{"="*30}\n{result}\n')
        print(f'Saved: {result_path}')

Saved: results/round2/template4_science_text1_theory_1_20250703_150648.txt
Saved: results/round2/template4_science_text1_theory_2_20250703_150721.txt
Saved: results/round2/template4_science_text1_theory_3_20250703_150842.txt
Saved: results/round2/template4_science_text2_theory_1_20250703_150926.txt
Saved: results/round2/template4_science_text2_theory_2_20250703_151023.txt
Saved: results/round2/template4_science_text2_theory_3_20250703_151050.txt
Saved: results/round2/template4_science_text3_theory_1_20250703_151134.txt
Saved: results/round2/template4_science_text3_theory_2_20250703_151227.txt
Saved: results/round2/template4_science_text3_theory_3_20250703_151317.txt
